In [20]:
#from langchain_openai import ChatOpenAI
#import openai
import os
import getpass
import tiktoken
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
#from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [21]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

In [9]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [12]:
loader = CSVLoader(file_path='../data/dur.csv')
data = loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100, length_function=tiktoken_len)
texts = text_splitter.split_documents(data)

In [14]:
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
#save to disk
emed_db = Chroma.from_documents(texts, hf,persist_directory="../db/dur_jhgan")

In [16]:
#벡터db의 데이터가 in-memory가 아니라 persistent storage인 disk에 저장되게 선언
emed_db.persist()
emed_db = None

In [17]:
# load from disk
emed_db = Chroma(persist_directory="./e약은요db_jhgan_의약품명추가",embedding_function=hf)

In [22]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceHub
from transformers import AutoTokenizer, AutoModelForCausalLM



In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

retriever = emed_db.as_retriever(search_type="similarity", search_kwargs={'k':3})

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

#답변 생성
qa_system_prompt = """You are an assistant for question-answering tasks. \
ONLY USE the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

# chat history 관리
store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
    output_messages_key= "answer",
)


# Test 수행

In [24]:
import pandas as pd

In [25]:
id = "0"
query1 = "명인아캄프로세이트정 노인이 먹어도 돼?"
conversational_rag_chain.invoke(
    {"input": query1},
    config={
        "configurable": {"session_id":id}
    },
)["answer"]

HfHubHTTPError:  (Request ID: EG09_Xs4R4QQ2w2UeaJgI)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/yanolja/KoSOLAR-10.7B-v0.2.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model yanolja/KoSOLAR-10.7B-v0.2 is too large to be loaded automatically (21GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [ ]:
dur_question = pd.read_csv("../data/.csv") 
dur_question.head()

### 참고사항
추후에 특정 의약품에 대한 질문을 이어서했을때의 정확도를 살피고자 할때는 session_id로 제공되는 부분에 의약품명을 넣으면 된다.

의약품명을 넣는 방법은 애초에 데이터셋을 만들때 의약품명 열을 하나 추가하면 된다

In [ ]:
import time
def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} 실행 시간: {elapsed_time} 초")
        return result
    return wrapper

In [ ]:
@measure_time
def invoke(id, input):
    answer = conversational_rag_chain.invoke(
        {"input": input},
        config={
            "configurable": {"session_id":id}
        },
    )["answer"]

    return answer

In [ ]:
answer = [] #llm의 응답을 저장할 리스트
for i, row in test_data.iterrows():
    answer.append(invoke(row.question, row.question))
    print(i+1,'번째 답변 : ',answer[i])

In [ ]:
question = test_data['question']
test_output = pd.DataFrame({"question": question,
                            "answer" : answer})
test_output.to_excel("e약은요 4차 테스트 결과_jhgan임베딩_문서3개.xlsx",index = False)

### db 초기화
```python
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()
```